In [5]:
import tkinter as tk
import random
import string
import sqlite3
import pandas as pd

#connect to database for password manager
sql_connect = sqlite3.connect('password_manager.db')
cur = sql_connect.cursor() # creates a cursor object 

cur.execute('''
CREATE TABLE IF NOT EXISTS passwords (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
        website TEXT,
        username TEXT,
        password TEXT
    )
''')
sql_connect.commit()

#Function to generate the secure password
def password_generator():
    password = ''.join(random.choice(string.ascii_letters + string.digits + string.punctuation) for _ in range(12))
    password_entry.delete(0, tk.END)
    password_entry.insert(0, password)

# Function to save password to the database
def save_password():
    website = website_entry.get()
    username = username_entry.get()
    password = password_entry.get()

    if website and username and password:
        cur.execute('INSERT INTO passwords (website, username, password) VALUES (?, ?, ?)', (website, username, password))
        sql_connect.commit()
        outcome_label.config(text="Password saved successfully!", fg="green")
    else:
        outcome_label.config(text="Missing Field! Please fill in all fields!", fg="red")

# Function to see all saved passwords that have been stored
def see_passwords():
    cur.execute('SELECT website, username, password FROM passwords')
    records = cur.fetchall()
    result_text = "\n".join([f"Website: {record[0]}, Username: {record[1]}, Password: {record[2]}" for record in records])
    view_window = tk.Toplevel(main)
    view_window.title("Stored Passwords")
    tk.Label(view_window, text=result_text, font=("Arial", 12), padx=10, pady=10).pack()

# Function to print passwords in Jupyter 
def print_passwords():
    pp = pd.read_sql_query('''
    SELECT * FROM passwords;
    ''', sql_connect)
    print(pp)  

# Main window
main = tk.Tk()
main.title("Secure Password Manager")

# Labels and Entries
tk.Label(main, text="Website:", font=("Arial", 12), padx=10, pady=10).grid(row=0, column=0, sticky="e")
website_entry = tk.Entry(main, width=30)
website_entry.grid(row=0, column=1, padx=10, pady=5)

tk.Label(main, text="Username:", font=("Arial", 12), padx=10, pady=10).grid(row=1, column=0, sticky="e")
username_entry = tk.Entry(main, width=30)
username_entry.grid(row=1, column=1, padx=10, pady=5)

tk.Label(main, text="Password:", font=("Arial", 12), padx=10, pady=10).grid(row=2, column=0, sticky="e")
password_entry = tk.Entry(main, width=30)
password_entry.grid(row=2, column=1, padx=10, pady=5)

# Buttons
generate_button = tk.Button(main, text="Generate Password", command=password_generator, bg="lightblue")
generate_button.grid(row=3, column=0, padx=10, pady=10)

save_button = tk.Button(main, text="Save Password", command=save_password, bg="lightgreen")
save_button.grid(row=3, column=1, padx=10, pady=10)

view_button = tk.Button(main, text="View Passwords", command=see_passwords, bg="lightyellow")
view_button.grid(row=4, column=0, columnspan=2, pady=10)

jupyter_print_button = tk.Button(main, text="View in Jupyter", command=print_passwords, bg="lightgray")
jupyter_print_button.grid(row=5, column=0, columnspan=2, pady=10)


# Outcome Label
outcome_label = tk.Label(main, text="", font=("Arial", 12), fg="red")
outcome_label.grid(row=6, column=0, columnspan=2, pady=10)

main.mainloop()

# Close the database connection when the app is closed
sql_connect.close()